In [1]:
#載入所需套件
import os
import numpy as np 
#import pandas as pd
#import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
import sys

In [2]:
# 資料所在路徑
dir_data = './data/kaggle_dogcat/'
# 影像大小
imagesize = (224, 224)
# 影像類別數
classnum = 2

# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
batchsize = 8
# 凍結網路層數
freezelayers = 2
# Epoch 數
epochnum = 20
# 模型輸出儲存的檔案
weightsfinal = 'model-resnet50-ml-marathon-final.h5'

In [4]:
# 透過 data augmentation 產生訓練與驗證用的影像資料
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_batches = train_datagen.flow_from_directory(dir_data + '/train',
                                                  target_size=imagesize,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=batchsize)

valid_datagen = ImageDataGenerator()
valid_batches = valid_datagen.flow_from_directory(dir_data + '/test',
                                                  target_size=imagesize,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=batchsize)

# 輸出各類別的索引值
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

# 以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(imagesize[0],imagesize[1],3))
x = net.output
x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(classnum, activation='softmax', name='softmax')(x)

# 設定凍結與要進行訓練的網路層
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:freezelayers]:
    layer.trainable = False
for layer in net_final.layers[freezelayers:]:
    layer.trainable = True

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])

# 輸出整個網路結構
print(net_final.summary())

# 訓練模型
net_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // batchsize,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // batchsize,
                        epochs = epochnum)

# 儲存訓練好的模型
net_final.save(weightsfinal)

Found 4000 images belonging to 2 classes.
Found 0 images belonging to 0 classes.
Class #0 = cats
Class #1 = dogs
Instructions for updating:
Colocations handled automatically by placer.


D:\Program Files\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 35s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalizatio

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
499/500 [============================>.] - ETA: 17s - loss: 0.7917 - acc: 0.7533

IndexError: list index out of range

### Testing

In [5]:
# 從參數讀取圖檔路徑
files = sys.argv[1:]

# 載入訓練好的模型
net = load_model('model-resnet50-final.h5')

cls_list = ['cats', 'dogs']

# 辨識每一張圖
for f in files:
    img = image.load_img(f, target_size=(224, 224))
    if img is None:
        continue
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    pred = net.predict(x)[0]
    top_inds = pred.argsort()[::-1][:5]
    print(f)
    for i in top_inds:
        print('    {:.3f}  {}'.format(pred[i], cls_list[i]))

OSError: Unable to open file (unable to open file: name = 'model-resnet50-final.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)